In [3]:
!pip install torch_geometric torch


  Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 17.9 MB/s eta 0:00:00a 0:00:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [1]:
import pandas as pd
from datetime import datetime
import torch
import torch.nn as nn
from torch_geometric.nn import HeteroConv, GATConv, SAGEConv
from torch_geometric.data import HeteroData

In [2]:
# Load the HeteroData object from the file
data = torch.load('hetero_data_final.pt')

In [3]:
class GNNModel(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super(GNNModel, self).__init__()
        self.conv = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('user', 'has_searched', 'destination'): GATConv((-1, -1), hidden_dim, add_self_loops=False),
                ('destination', 'has_hotel', 'hotel'): SAGEConv((-1, -1), hidden_dim),
            }, aggr='sum')
            self.conv.append(conv)

    def forward(self, data):
        for conv in self.conv:
            x_dict = {key: value.to(torch.float32) for key, value in data.x_dict.items()}
            edge_index_dict = {key: value.to(torch.long) for key, value in data.edge_index_dict.items()}
            x_dict = conv(x_dict, edge_index_dict)
        return x_dict['hotel']

In [4]:
class RecommendationHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RecommendationHead, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

In [5]:
def construct_query_graph(user_id, destination_id, checkin_date, checkout_date, room_cnt, adult_cnt, children_cnt):
    query_data = HeteroData()
    query_data['user'].x = data['user'].x
    query_data['destination'].x = data['destination'].x
    query_data['hotel'].x = data['hotel'].x

    checkin_date = int(datetime.strptime(checkin_date, '%Y-%m-%d').timestamp())
    checkout_date = int(datetime.strptime(checkout_date, '%Y-%m-%d').timestamp())
    edge_attr = torch.tensor([[room_cnt, adult_cnt, children_cnt, checkin_date, checkout_date]])
    edge_index = torch.tensor([[user_id, destination_id]])
    query_data['user', 'has_searched', 'destination'].edge_index = torch.tensor(edge_index, dtype=torch.int64).T
    query_data['user', 'has_searched', 'destination'].edge_attr = torch.tensor(edge_attr, dtype=torch.int64)

    l = []
    for i, d in enumerate(data['destination', 'has_hotel', 'hotel']['edge_index'][0]):
        if d == destination_id:
            l.append([d, data['destination', 'has_hotel', 'hotel']['edge_index'][1][i]])
    l = torch.tensor(l).T

    query_data['destination', 'has_hotel', 'hotel'].edge_index = l

    h = []
    for i, u in enumerate(data['user', 'has_booked', 'hotel']['edge_index'][0]):
        if u == user_id:
            h.append([u, data['user', 'has_booked', 'hotel']['edge_index'][1][i]])
    h = torch.tensor(h).T

    query_data['user', 'has_booked', 'hotel'].edge_index = h

    return query_data

In [6]:
def recommend_hotels(user_id, destination_id, checkin_date, checkout_date, room_cnt=1, adult_cnt=1, children_cnt=0):
    gnn_model = GNNModel(8, 8)  # New instance of the model
    recommendation_head = RecommendationHead(8, 1)  # New instance of the recommendation head

    gnn_model.eval()
    recommendation_head.eval()

    query_graph = construct_query_graph(user_id, destination_id, checkin_date, checkout_date, room_cnt, adult_cnt, children_cnt)

    with torch.no_grad():
        node_representations_target = gnn_model(query_graph)
        predictions = recommendation_head(node_representations_target)

    k = 7
    top_predictions, top_indices = torch.topk(predictions, k, dim=0, largest=True)
    return top_predictions, top_indices

In [7]:
destination_id = 54
user_id = 820
checkin_date = "2023-06-13"
checkout_date = "2023-06-15"
room_cnt = 2
adult_cnt = 2
children_cnt = 0
top_predictions, top_indices = recommend_hotels(user_id, destination_id, checkin_date, checkout_date, room_cnt, adult_cnt, children_cnt)

/Users/ravranja6/opt/anaconda3/envs/langchain-1/lib/python3.11/site-packages/torch_geometric/nn/conv/hetero_conv.py:76: UserWarning: There exist node types ({'user'}) whose representations do not get updated during message passing as they do not occur as destination type in any edge type. This may lead to unexpected behavior.
  warnings.warn(
/var/folders/tb/hsmdnsw10_xgwhg6wc11dxgm0000gn/T/ipykernel_93823/1377356286.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_data['user', 'has_searched', 'destination'].edge_index = torch.tensor(edge_index, dtype=torch.int64).T
/var/folders/tb/hsmdnsw10_xgwhg6wc11dxgm0000gn/T/ipykernel_93823/1377356286.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(s

In [8]:
print(top_indices)
rec_hotel = top_indices.tolist()
rec_hotel = [i[0] for i in rec_hotel]
print(rec_hotel)

tensor([[1008],
        [1011],
        [1009],
        [1012],
        [1010],
        [1014],
        [3452]])
[1008, 1011, 1009, 1012, 1010, 1014, 3452]


In [14]:
lst = []
for i, d in enumerate(data['destination', 'has_hotel', 'hotel']['edge_index'][0]):
    if d == 54:
        lst.append([d, data['destination', 'has_hotel', 'hotel']['edge_index'][1][i]])

print(lst)

[[tensor(54), tensor(57)]]
